First contact
===

Now that we have a Redis server running and we have compiled PaGMO and PyGMO with ZeroMQ support (see The Setup (TODO: link)), we can run an example program that leverages the functionality of this type of island.


In [1]:
from PyGMO import *
import time

prob = problem.schwefel(10)
algo = algorithm.de(10)
pop = population(prob, 20)

isl = zmq_island(algo, pop)
isl.set_broker_details("127.0.0.1", 6379)
isl.set_token("schwefel10_de10_pop20")
isl.set_ip("127.0.0.1")

def evolve():
    isl.evolve(10)
    print("Best: ", isl.population.champion.x)

    time.sleep(1)

This code will set up a single ZeroMQ island working on the 10-dimensional Schwefel problem and using the Differential Evolution algorithm. The population of the island will be 20. Lines 4-6 set up the problem, and lines 8-12 set up the island itself. 

In this tutorial we assume that the broker is at 127.0.0.1:6379, and we will be binding the receiving port only on the loopback inteface, but in a networked setup you would use IPs that are accessible beyond localhost.

The channel token is set to `schwefel10_de10_pop20`, which can be any string, but choosing a token relevant to the actual properties of the computation will help you determine which group of islands is working on what problem. If we start this program, it will connect to the broker and advertise itself as an island working on that token, but because there are no other peers it will simply perform the evolution locally. This is a typical output:


In [2]:
evolve()

Best:  (424.4920569537634, 440.7206390429582, 399.43102290788795, -299.52078183929916, 424.4274999225018, 395.96019766970727, 396.61431073790675, 414.7987356103403, -293.15767979233385, -152.01178644903854)


Note that because the island is not actually connected to the ZeroMQ network yet, it's not exchanging solutions with peers.

Now, if we call the `connect()` function on this island, we'll be ready to accept migrants and send our own as well.

In [3]:
isl.connect()

True

Because IPython doesn't have an easy way of executing two threads and displaying the result easily, I'll start up another program just like this outside IPython and call evolve() here -- it should receive the population from the other program, and display it.

In [5]:
evolve()

Best:  (420.9687728816528, 420.9687569400687, 420.9687642651889, 420.968743361018, 420.96875116870063, 420.968771074309, 420.96871292112354, 420.9687127857178, 420.96875246113143, 420.9687562620334)


As you can see, our current solution is much better than the original and because the program in the background had been running for a little while before that, and it had already converged on the optimum for the Schwefel function.

Here's a snippet of the output on the other side: 

    Best:  (420.96880347940544, 420.9687194811639, 420.96880839006843, 420.9687540217262, 420.9687810359755, 420.9687609702579, 420.968730572192, 420.96875630500364, 420.9687648447646, 420.96880903640675)
    Best:  (420.9687351908159, 420.96872163933637, 420.9687399639167, 420.9687069931382, 420.9686997709129, 420.9687976850297, 420.9687312413354, 420.9687104977846, 420.9687392971252, 420.9687677879039)
    DEBUG: Opening connection to 127.0.0.1:2609
    Best:  (420.9687701356386, 420.9687781629428, 420.9687691217583, 420.9687483448487, 420.96878827364174, 420.96875668743473, 420.96870707941497, 420.9687079359776, 420.96874266075383, 420.96872310305565)
    Best:  (420.9687728816528, 420.9687569400687, 420.9687642651889, 420.968743361018, 420.96875116870063, 420.968771074309, 420.96871292112354, 420.9687127857178, 420.96875246113143, 420.9687562620334)
    Best:  (420.9687533680281, 420.96876243183203, 420.9687590126924, 420.9687372408334, 420.96875017480545, 420.96877891439186, 420.9687558066539, 420.9687460272027, 420.9687420838824, 420.9687647092545)


Note how it opens a connection to our island as soon as we call `isl.connect()`. Then it sent its population, which had already converged on the answer and was better than our original value.